In [16]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [17]:
x_min = -5.0
x_max = 15.0

N = 500
t_max = 10.0
a = 1.0

In [18]:
def FTCS(nu:np.double):

    dx = (x_max - x_min) / (N-1)
    dt = min( (0.75 * (dx**2)) / (2*nu), (0.75*dx)/a )   

    # Initial condition
    x = np.linspace(x_min, x_max, N)
    u_max = 10
    u = u_max * np.exp(-x*x*100)

    t = 0.0
    while t < t_max:

        if t > t_max - dt:
            dt = t_max - t
        t = t + dt

        u_cache = u.copy()
 
        # Periodic Boundary condition
        u_cache[0] = u[0] - (a*dt)/(2*dx) * (u[1] - u[-2]) + (nu*dt)/(dx**2) * (u[1] - 2 * u[0] + u[-2])
        u_cache[-1] = u_cache[0]
        for i in range(1, N-1):
            # FTCS scheme
            u_cache[i] = u[i] - (a*dt)/(2*dx) * (u[i+1] - u[i-1]) + (nu*dt)/(dx**2) * (u[i+1] - 2 * u[i] + u[i-1])
        u = u_cache

    return u

In [19]:
def upwind(nu:np.double):
    dx = (x_max - x_min) / (N-1)
    dt = min( (0.75 * (dx**2)) / (2*nu), (0.75*dx)/a )   

    # Initial condition
    x = np.linspace(x_min, x_max, N)
    u_max = 10
    u = u_max * np.exp(-x*x*100)

    t = 0.0
    while t < t_max:

        if t > t_max - dt:
            dt = t_max - t
        t = t + dt

        u_cache = u.copy()
        u_cache[0] = u[0] - (a*dt)/dx * (u[0] - u[-2]) + (nu*dt)/(dx**2) * (u[1] - 2 * u[0] + u[-2])
        u_cache[-1] = u_cache[0]
        for i in range(1, N-1):
            # FTBS scheme
            u_cache[i] = u[i] - (a*dt)/dx * (u[i] - u[i-1]) + (nu*dt)/(dx**2) * (u[i+1] - 2 * u[i] + u[i-1])
        u = u_cache

    return u

In [20]:
x_test = np.linspace(x_min, x_max, N)

In [30]:
def draw(param):
    FTCS_sol = FTCS(param)
    upwind_sol = upwind(param)

    fig, axes = plt.subplots(1, 2, figsize=(12,5))
    axes[0].plot(x_test, FTCS_sol, '-')
    axes[0].set_title("FTCS scheme")
    axes[0].set_xlabel("x")
    axes[0].set_ylabel("v(x,t)")

    axes[1].plot(x_test, upwind_sol, '-')
    axes[1].set_title("FTBS scheme")
    axes[1].set_xlabel("x")
    axes[1].set_ylabel("v(x,t)")

    plt.tight_layout()
    plt.show()

param_slider = widgets.FloatSlider(value=0.05, min=0.001, max=0.05, step=0.001, description="Coefficient",
                                   readout_format='.3f', style={'description_width': 'initial'})

interact(draw, param=param_slider)

interactive(children=(FloatSlider(value=0.05, description='Coefficient', max=0.05, min=0.001, readout_format='…

<function __main__.draw(param)>

We observe that FTCS scheme breaks when coefficient is about 0.016 and below. Upwind FTBS scheme breaks when coefficient is larger than 0.007, and indeed the ability of the scheme for small diffusion coefficient.

We can compute the symbol of FTCS scheme is 

$1-4r \sin^2(\frac{\xi}{2})-i*2s \sin(\xi)$,

where $s=\frac{adt}{2dx}=0.375$, $r=\frac{\nu dt}{dx^2}=18.75\nu$ 
For small r this norm will be larger than 1, causing unstability. For $\nu$ larger than 0.02 the min step for $\Delta t$ ensures there's no r larger than 0.375.

The symbol of upwind scheme is 

$1-4(r+s) \sin^2(\frac{\xi}{2})-i*2s \sin(\xi)$,

since there exists a large s in the real part, only very small r can have stable scheme.